In [1]:
# VGG

import os
import torch
import import_ipynb
import numpy as np
from copy import deepcopy
from Network import Network
from HCPDataLoader import getHCPDataSet
from tensorboardX import SummaryWriter
import matplotlib.pyplot as plt
from ResultCalculator import resultCalculator
from LoggerCreator import loggerCreator
from EarlyStopping import EarlyStopping
from torch.utils.data import Dataset, DataLoader

os.system('rm -rf ./logs/*')
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
logger = loggerCreator()

importing Jupyter notebook from Network.ipynb
importing Jupyter notebook from HCPDataLoader.ipynb
importing Jupyter notebook from ResultCalculator.ipynb
importing Jupyter notebook from LoggerCreator.ipynb
importing Jupyter notebook from EarlyStopping.ipynb


In [2]:
HCPDataListPath = './s2_con1_label.txt'
HCPDataRootPath = '../../data/disk/'

BATCHSIZE = 16
LEARNINGRATE = 0.0001
EPOCH = 50
EXPERIMENTTIMES = 10
PATIENCE = 10

In [3]:
dataSet = getHCPDataSet(HCPDataRootPath, HCPDataListPath, evalRate=0.05, testRate=0.15)

Training: 2644, Evaluation: 165, Test: 496


In [4]:
def writeAcc(idx, loss, predict, trainLabel, writer, prefix=None, outputPath=None):
    acc, result = resultCalculator(predict, trainLabel, prefix=prefix, outputPath=outputPath)
    logger.info("{}: ({}) acc:{}%, loss:{}".format(idx, prefix, acc, loss))
    
    writer.add_scalar('acc', acc, idx)
    writer.add_scalar('loss', loss, idx)
    
    resultSum = result.sum(1)
    for i in range(result.shape[0]):
        writer.add_scalar('tasks/{}'.format(i), result[i][i]/resultSum[i] * 100, idx)

In [5]:
def runAnEpoch(dataLoader, net, lossFunc):
    loss = 0
    predicts = None
    labels = None
    
    net.eval()
    for idx, (data, label) in enumerate(dataLoader):
        labels = label if loss == 0 else np.hstack((labels, label))
        data = data.cuda()
        label = label.cuda()
        predict = net(data)
        
        predicts = predict.cpu().detach().numpy() if loss == 0 else np.vstack((predicts, predict.cpu().detach().numpy()))
        loss += lossFunc(predict, label).item()
        
        logger.info("Evaluationg, {}".format(labels.shape[0]))
    net.train()
    return  predicts, labels, loss/labels.shape[0]

In [6]:
def trainNetwork(learningRate, BatchSize, experimentTimes = 0):
    logger.info("--------------------------------------------------------------")
    logger.info("Network loading, learningRate: {}, BatchSize: {}, experimentTimes: {}"
                .format(learningRate, BatchSize, experimentTimes))
    trainWriter = SummaryWriter('logs/train_{}_{}_{}/'
                                .format(BatchSize, learningRate, experimentTimes))
    evalWriter = SummaryWriter('logs/eval_{}_{}_{}/'
                               .format(BatchSize, learningRate, experimentTimes))
    testWriter = SummaryWriter('logs/test_{}_{}_{}/'
                               .format(BatchSize, learningRate, experimentTimes))
    
    trainLoader = DataLoader(dataSet.trainDataSet, batch_size = BatchSize, num_workers=16, shuffle = True)
    evalLoader = DataLoader(dataSet.evalDataSet, batch_size = BatchSize, num_workers=8, shuffle = True)
    testLoader = DataLoader(dataSet.testDataSet, batch_size = BatchSize, num_workers=8, shuffle = True)
    logger.info("DataLoader have been created!")
    
    net = Network().cuda()
    logger.info("Network have been established.")

    lossFunc = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=LEARNINGRATE)
    early_stopping = EarlyStopping(PATIENCE, verbose=True, experimentTimes=experimentTimes)
    net.train()
    logger.info("Train begining.")
    idx = 0
    
    for epoch in range(EPOCH):
        try:
            for trainData, trainLabel in trainLoader:
                idx += 1
                trainData = trainData.cuda()
                trainLabel = trainLabel.cuda()

                predict = net(trainData)
                loss = lossFunc(predict, trainLabel)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                if idx % 5 == 0:
                    writeAcc(idx, loss.item(), predict, trainLabel, trainWriter, 'train')

                if idx % 50 == 0:
                    logger.info("Evaluating!")
                    predicts, label, loss = runAnEpoch(evalLoader, net, lossFunc)
                    writeAcc(idx, loss, predicts, label, evalWriter, 'eval', outputPath='./RESULT.md')

                    early_stopping(loss, net)
                    if early_stopping.early_stop:
                        break

        except Exception as exc:
            logger.error(str(exc))
            
        if early_stopping.early_stop:
            break
        
    if early_stopping.early_stop:
        logger.warning("Early stopping!")
        predicts, label, loss = runAnEpoch(testLoader, net, lossFunc)
        writeAcc(idx, loss, predicts, label, testWriter, 'test', outputPath='./testRESULT.md')
     
    trainWriter.close()
    evalWriter.close()

In [ ]:
if __name__ == "__main__":
    for experimentTimes in range(EXPERIMENTTIMES):
        trainNetwork(LEARNINGRATE, BATCHSIZE, experimentTimes)

2020-06-04 18:51:46,303[line:2] - INFO: --------------------------------------------------------------
2020-06-04 18:51:46,306[line:4] - INFO: Network loading, learningRate: 0.0001, BatchSize: 16, experimentTimes: 0
2020-06-04 18:51:46,312[line:15] - INFO: DataLoader have been created!
